# P1. Vaccines for everyone?

### The COVID-19 vaccination campaign in the United States and its demonstrated effectiveness in targeting vulnerable minorities 
---
### Epidemiological Data Analysis: US Vaccination Campaign (November 2020 - March 2021)

The scope of this study is to determine the effectiveness of the US vaccination campaign in attending the population that is most affected by covid-19 disease. This is achieved by revising the total number of doses applied, vaccination coverage, population demographics such as gender, race, socioeconomic status, and education, versus epidemiologic variables: incidence, prevalence, hospitalization, UCI, death numbers.

## Data Collection & Cleanup

In [1]:
## Dependencies
import pandas as pd
import numpy as np
import requests
import time
import datetime

# Import App Token
from app_tokens import cdc_token

## 1. Covid-19 Vaccination Data
### 1.1 Data Retrieval

In [2]:
## Import data from CSV
hesitancy_df = pd.read_csv('input_data/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv' )#, encoding='latin-1')

In [3]:
## Select data fields to be used
columns = [
    'FIPS Code',
    'Social Vulnerability Index (SVI)',
    'SVI Category',
    'Percent adults fully vaccinated against COVID-19',
    'Percent Hispanic',
    'Percent non-Hispanic American Indian/Alaska Native',
    'Percent non-Hispanic Asian',
    'Percent non-Hispanic Black',
    'Percent non-Hispanic Native Hawaiian/Pacific Islander',
    'Percent non-Hispanic White'
]

## Build sorted DataFrame & Reset index
vaccination_df = hesitancy_df[columns].sort_values('FIPS Code')
vaccination_df.reset_index(inplace=True, drop=True)

## Compute "non-Hispanic Other"
vaccination_df['Percent non-Hispanic Other'] = 1 - vaccination_df.iloc[:,4:10].sum(axis=1)

fips = []
for index, row in vaccination_df.iterrows():
        fips.append(f"{row['FIPS Code']:0>5}")

vaccination_df['FIPS Code'] = fips

## Set FIPS Code as index
vaccination_df.set_index('FIPS Code', drop=True, inplace=True)

## Export DataFrame to CSV
vaccination_df.to_csv('analysis_data/vaccination_data.csv')

## DataFrame preview
vaccination_df.head()

,Social Vulnerability Index (SVI),SVI Category,Percent adults fully vaccinated against COVID-19,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Percent non-Hispanic Other
FIPS Code,,,,,,,,,,
01001,0.44,Moderate Vulnerability,0.114,0.0283,0.0025,0.0103,0.1900,0.0001,0.7460,0.0228
01003,0.22,Low Vulnerability,0.176,0.0456,0.0065,0.0092,0.0917,0.0000,0.8307,0.0163
01005,1.00,Very High Vulnerability,0.128,0.0436,0.0029,0.0048,0.4744,0.0000,0.4581,0.0162
01007,0.60,High Vulnerability,0.115,0.0257,0.0013,0.0012,0.2214,0.0000,0.7453,0.0051
01009,0.42,Moderate Vulnerability,0.095,0.0926,0.0007,0.0037,0.0153,0.0004,0.8689,0.0184


## 2. US Census Reference (2019): Population (by county)
### 2.1 Data Retrieval

In [4]:
# In order to build a useful Census Data set, we need to import the original csv file.
# We used the encoding 'latin-1' to code the characters into the Latin alphabet. 

census_df = pd.read_csv('input_data/co-est2019-alldata.csv' )#, encoding='latin-1')

In [5]:
# We now select the columns we will be using: 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2019'

census_2019 = census_df[['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2019']]

# DataFrame Preview
census_2019.head()

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019
0,1,0,Alabama,Alabama,4903185
1,1,1,Alabama,Autauga County,55869
2,1,3,Alabama,Baldwin County,223234
3,1,5,Alabama,Barbour County,24686
4,1,7,Alabama,Bibb County,22394


In [6]:
# Once we have selected the columns, we need to clean the data set from empty values, NAN's.
# Since the data set has state totals, we create two dataframes, one with the total information by state
# and another with information by county. 

state_2019 = census_2019.loc[census_2019['COUNTY']==0]
state_2019.reset_index(inplace=True, drop=True)

# DataFrame Preview
state_2019.head()

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019
0,1,0,Alabama,Alabama,4903185
1,2,0,Alaska,Alaska,731545
2,4,0,Arizona,Arizona,7278717
3,5,0,Arkansas,Arkansas,3017804
4,6,0,California,California,39512223


In [7]:
# We also dropped the total row, since we want the information by county, not by state. 

county_2019 = census_2019.drop(census_2019.index[census_2019["COUNTY"]==0])

In [8]:
# We select the FIPS code to use as index and as the merging point.
# Since in the county dataframe, the fips is disaggregated, we need to go through the rows and create the FIPS code.

fips = []
for index, row in  county_2019.iterrows():
    fips.append(f'{row["STATE"]:>02}{row["COUNTY"]:>03}')

# We drop the columns we used to make the FIPS code and create another for the code. 

county_2019.insert(0, 'FIPS Code', fips)
county_2019.set_index('FIPS Code', drop=True, inplace=True)
county_2019.drop(labels=['STATE', 'COUNTY'], axis=1, inplace=True)

# DataFrame Preview
county_2019

,STNAME,CTYNAME,POPESTIMATE2019
FIPS Code,,,
01001,Alabama,Autauga County,55869
01003,Alabama,Baldwin County,223234
01005,Alabama,Barbour County,24686
01007,Alabama,Bibb County,22394
01009,Alabama,Blount County,57826
...,...,...,...
56037,Wyoming,Sweetwater County,42343
56039,Wyoming,Teton County,23464
56041,Wyoming,Uinta County,20226


## 3. US Census Reference (2019): Sex, Age & Ethnicity (by county)
### 3.1 Data Retrieval

In [9]:
# We need the demographic information of the counties for comparing it with the vaccionation campaign.
# We import the census data set, also using the 'latin-1' encoding. 

all_data = pd.read_csv("input_data/cc-est2019-alldata.csv" )#, encoding='latin-1')

In [10]:
# Since the age classification clashes with the other data sets, we are reclassifying it. 
# We use numpy select to creat a new column following the new age groups conditions. 

age_conditions = [
    (all_data['AGEGRP'] == 0),
    (all_data['AGEGRP'] >= 1) & (all_data['AGEGRP'] <= 4),
    (all_data['AGEGRP'] >= 5) & (all_data['AGEGRP'] <= 10),
    (all_data['AGEGRP'] >= 11) & (all_data['AGEGRP'] <= 13),
    (all_data['AGEGRP'] >= 14) & (all_data['AGEGRP'] <= 18) 
]

age_values = ['0', '0 - 17 years', '18 - 49 years', '50 - 64 years', '65 + years']

all_data['Age_group'] = np.select(age_conditions, age_values)

In [11]:
# If age group is 0, it means the the sum of all the age. We don't need it, so we select it. 

all_data_tot = all_data.loc[all_data['AGEGRP']==0]

In [12]:
# Since we are focusing in the last year, we must select the year we need.
# We use a conditional loc. 

all_data_tot = all_data_tot.loc[all_data['YEAR']==12]
all_data_tot.reset_index(inplace=True, drop=True)

In [13]:
# We create the FIPS code again. 

fips_all_data = []
for index, row in  all_data_tot.iterrows():
    fips_all_data.append(f'{row["STATE"]:>02}{row["COUNTY"]:>03}')
    
all_data_tot.insert(0, 'FIPS Code', fips_all_data)
all_data_tot.set_index('FIPS Code', drop=True, inplace=True)

all_data_tot = all_data_tot.drop(labels=['STATE', 'COUNTY'], axis=1)

In [14]:
# We select the demograhics we are going to use from our census dataframe.

census_2019_sex = all_data_tot[['TOT_MALE','TOT_FEMALE']]

# DataFrame Preview
census_2019_sex

,TOT_MALE,TOT_FEMALE
FIPS Code,,
01001,27092,28777
01003,108247,114987
01005,13064,11622
01007,11929,10465
01009,28472,29354
...,...,...
56037,21808,20535
56039,12142,11322
56041,10224,10002


###### County_2019 dataframe with age group totals

In [15]:
# We create the datafram2 with the age group.

all_data_age = all_data[['STATE', 'COUNTY', 'YEAR', 'TOT_POP', 'Age_group']]

In [16]:
# We drop the sum total row. 

all_data_age = all_data_age.drop(all_data_age.index[all_data_age["Age_group"]=='0'])

In [17]:
# We select the year again. 

all_data_age = all_data_age.loc[all_data_age['YEAR']==12]

In [18]:
# We create the fips and drop the columns we no longer need.

fips_age = []
for index, row in  all_data_age.iterrows():
    fips_age.append(f'{row["STATE"]:>02}{row["COUNTY"]:>03}')
    
all_data_age.insert(0, 'FIPS Code', fips_age)
all_data_age.set_index('FIPS Code', drop=True, inplace=True)
all_data_age.drop(labels=['STATE', 'COUNTY'], axis=1, inplace=True)

In [19]:
# We group the sum of people of certain age group by Fips code and age group.

age_group = pd.DataFrame(all_data_age.groupby(['FIPS Code','Age_group'])['TOT_POP'].sum())

In [20]:
# We can't really use the previous dataframe so we use unstack to have a dataframe we can merge.
# When we unstacked, we created a multiindex, so we use droplevel to retrieve the index we want. 

census_2019_age = age_group.unstack()
census_2019_age = census_2019_age['TOT_POP']

# DataFrame Preview
census_2019_age

Age_group,0 - 17 years,18 - 49 years,50 - 64 years,65 + years
FIPS Code,,,,
01001,14252,21652,11041,8924
01003,52268,77402,46734,46830
01005,5595,9477,4753,4861
01007,4992,9233,4436,3733
01009,14522,21002,11488,10814
...,...,...,...,...
56037,12049,16959,7846,5489
56039,4586,10694,4467,3717
56041,6215,7229,3757,3025


###### County_2019 dataframe for ethnicity

In [21]:
# To create a dataframe about ethinicity, we must select the information we need from the vaccination dataset.

census_2019_ethnicity = vaccination_df[[
    'Percent Hispanic',
    'Percent non-Hispanic American Indian/Alaska Native',
    'Percent non-Hispanic Asian',
    'Percent non-Hispanic Black',
    'Percent non-Hispanic Native Hawaiian/Pacific Islander',
    'Percent non-Hispanic White',
    'Percent non-Hispanic Other'
]]

In [22]:
# # We also create the fips code as index. 

# fips_ethnicity = []
# for index, row in  census_2019_ethnicity.iterrows():
#     row_string = f"{row['FIPS Code']:.0f}"
#     fips_ethnicity.append(f"{row_string:>05}")
    
# census_2019_ethnicity = census_2019_ethnicity.drop('FIPS Code', axis=1)
# census_2019_ethnicity.insert(0, 'FIPS Code', fips_ethnicity)
# census_2019_ethnicity.set_index('FIPS Code', drop=True, inplace=True)

In [23]:
# DataFrame Preview
census_2019_ethnicity

,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Percent non-Hispanic Other
FIPS Code,,,,,,,
01001,0.0283,0.0025,0.0103,0.1900,0.0001,0.7460,0.0228
01003,0.0456,0.0065,0.0092,0.0917,0.0000,0.8307,0.0163
01005,0.0436,0.0029,0.0048,0.4744,0.0000,0.4581,0.0162
01007,0.0257,0.0013,0.0012,0.2214,0.0000,0.7453,0.0051
01009,0.0926,0.0007,0.0037,0.0153,0.0004,0.8689,0.0184
...,...,...,...,...,...,...,...
56037,0.1588,0.0102,0.0074,0.0112,0.0003,0.7956,0.0165
56039,0.1503,0.0033,0.0125,0.0124,0.0012,0.8134,0.0069
56041,0.0913,0.0065,0.0016,0.0011,0.0000,0.8752,0.0243


In [24]:
# We save the dataframe as a csv to use in the analysis 

census_2019_ethnicity.to_csv('clean_data/census_2019_ethnicity.csv')

In [25]:
# We merge county, sex, age and ethnicity dataframes

census_2019_combined = county_2019
census_2019_combined = census_2019_combined.merge(census_2019_sex, how='inner', left_index=True, right_index=True)
census_2019_combined = census_2019_combined.merge(census_2019_age, how='inner', left_index=True, right_index=True)
census_2019_combined = census_2019_combined.merge(census_2019_ethnicity, how='inner', left_index=True, right_index=True)

In [26]:
# DataFrame Preview

census_2019_combined

,STNAME,CTYNAME,POPESTIMATE2019,TOT_MALE,TOT_FEMALE,0 - 17 years,18 - 49 years,50 - 64 years,65 + years,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Percent non-Hispanic Other
FIPS Code,,,,,,,,,,,,,,,,
01001,Alabama,Autauga County,55869,27092,28777,14252,21652,11041,8924,0.0283,0.0025,0.0103,0.1900,0.0001,0.7460,0.0228
01003,Alabama,Baldwin County,223234,108247,114987,52268,77402,46734,46830,0.0456,0.0065,0.0092,0.0917,0.0000,0.8307,0.0163
01005,Alabama,Barbour County,24686,13064,11622,5595,9477,4753,4861,0.0436,0.0029,0.0048,0.4744,0.0000,0.4581,0.0162
01007,Alabama,Bibb County,22394,11929,10465,4992,9233,4436,3733,0.0257,0.0013,0.0012,0.2214,0.0000,0.7453,0.0051
01009,Alabama,Blount County,57826,28472,29354,14522,21002,11488,10814,0.0926,0.0007,0.0037,0.0153,0.0004,0.8689,0.0184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,Wyoming,Sweetwater County,42343,21808,20535,12049,16959,7846,5489,0.1588,0.0102,0.0074,0.0112,0.0003,0.7956,0.0165
56039,Wyoming,Teton County,23464,12142,11322,4586,10694,4467,3717,0.1503,0.0033,0.0125,0.0124,0.0012,0.8134,0.0069
56041,Wyoming,Uinta County,20226,10224,10002,6215,7229,3757,3025,0.0913,0.0065,0.0016,0.0011,0.0000,0.8752,0.0243


In [27]:
# We save that census as a csv. 

census_2019_combined.to_csv('clean_data/census_2019_combined.csv')

## 4. COVID-19 Case Surveillance
### 4.1 Data Retrieval
Collect *Case Surveillance* data from CDC Socrata API

In [28]:
## Set list of months to collect data from
months = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
          '2021-01', '2021-02', '2021-03', '2021-04']

## Define fields to be retrieved
fields = 'case_month, county_fips_code, hosp_yn, icu_yn, death_yn'
fips.append('NA')

## Build empty DataFrames & Set FIPS Code (plus NA) as Index
patients_df = pd.DataFrame(index=fips)
hospitalized_df = pd.DataFrame(index=fips)
icu_df = pd.DataFrame(index=fips)
death_df = pd.DataFrame(index=fips)

## Rename index
patients_df.index.rename('FIPS Code', inplace=True)
hospitalized_df.index.rename('FIPS Code', inplace=True)
icu_df.index.rename('FIPS Code', inplace=True)
death_df.index.rename('FIPS Code', inplace=True)

In [29]:
## Set query parameters
query_url = "https://data.cdc.gov/resource/n8mc-b4w4.json?"
params = {
    '$$app_token': cdc_token,
    '$limit': 25000000,
    '$offset': 0,
    '$select': fields
}

In [30]:
## If throttled by API, set individual month for manual loop:
months = ['2020-01']

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

## Retrieve Loop
for month in months:
    
    ## Print Log Status
    print(f"Processing Month: {month} [{datetime.datetime.now().strftime('%H:%M:%S')}]")
    
    ## Set month query
    params['case_month'] = month
    ## Retrieve month data & Store in DataFrame
    response_month = requests.get(query_url, params=params).json()
    response_df = pd.DataFrame(response_month)

    ## Process response values to build useful DataFrames
    patients_df[month] = response_df.groupby('county_fips_code')['case_month'].count()
    hospitalized_df[month] = response_df.loc[response_df['hosp_yn'] == 'Yes'].groupby('county_fips_code')['hosp_yn'].count()
    icu_df[month] = response_df.loc[response_df['icu_yn'] == 'Yes'].groupby('county_fips_code')['icu_yn'].count()
    death_df[month] = response_df.loc[response_df['death_yn'] == 'Yes'].groupby('county_fips_code')['death_yn'].count()

    ## Skip time.sleep() for last month in months
    if month != months[-1]:
        print("Sleeping...")
        time.sleep(60*61)

## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Month: 2020-01 [03:05:31]
------------------------------
Data Retrieval Complete
------------------------------


In [31]:
## IF MONTH-BY-MONTH MANUAL LOOP:

# ## 1. Export collected data to CSV

# patients_df.to_csv('clean_data/patients.csv')
# hospitalized_df.to_csv('clean_data/hospitalized.csv')
# icu_df.to_csv('clean_data/icu.csv')
# death_df.to_csv('clean_data/death.csv')

## 2. Change value for month & Run next loop

## 3. Import previously collected data from CSV

# patients_df = pd.read_csv('clean_data/patients.csv', dtype={'FIPS Code':str})
# patients_df.iloc[-1,0] = 'NA'
# patients_df.set_index('FIPS Code', drop=True, inplace=True)

# hospitalized_df = pd.read_csv('clean_data/hospitalized.csv', dtype={'FIPS Code':str})
# hospitalized_df.iloc[-1,0] = 'NA'
# hospitalized_df.set_index('FIPS Code', drop=True, inplace=True)

# icu_df = pd.read_csv('clean_data/icu.csv', dtype={'FIPS Code':str})
# icu_df.iloc[-1,0] = 'NA'
# icu_df.set_index('FIPS Code', drop=True, inplace=True)

# death_df = pd.read_csv('clean_data/death.csv', dtype={'FIPS Code':str})
# death_df.iloc[-1,0] = 'NA'
# death_df.set_index('FIPS Code', drop=True, inplace=True)

# ## 3. Manually add new data to existing DataFrames:

# month = '2020-10'
# patients_df[month] = patients_df1[month].values
# hospitalized_df[month] = hospitalized_df1[month].values
# icu_df[month] = icu_df1[month].values
# death_df[month] = death_df1[month].values

In [32]:
## Build summary DataFrame (total absolute values)
data_abs = {
    'Patients': patients_df.sum(axis=1),
    'Hospitalized': hospitalized_df.sum(axis=1),
    'ICU': icu_df.sum(axis=1),
    'Death': death_df.sum(axis=1)
}

total_abs = pd.DataFrame(data_abs)

In [33]:
## IF MONTH-BY-MONTH MANUAL LOOP:
## RUN SCRIPT ONLY AFTER COLLECTING ALL DATA

# ## Fill na values with zeros
# patients_df.fillna(0, inplace=True)
# hospitalized_df.fillna(0, inplace=True)
# icu_df.fillna(0, inplace=True)
# death_df.fillna(0, inplace=True)

# ## Export collected data to CSV
# patients_df.to_csv('analysis_data/patients.csv')
# hospitalized_df.to_csv('analysis_data/hospitalized.csv')
# icu_df.to_csv('analysis_data/icu.csv')
# death_df.to_csv('analysis_data/death.csv')
# total_abs.to_csv('analysis_data/total_abs.csv')

## 5. Geo-location coordinates (by county)

### 5.1 Data Retrieval

Import *United States Cities Database* CSV file (downloaded from https://simplemaps.com/data/us-cities)

In [34]:
## Import geo-location data CSV
county_data = pd.read_csv('input_data/uscities.csv', dtype={'county_fips':str})
## Select data fields
county_data = county_data[['county_fips','lat','lng']]
## Rename columns, sort values (by FIPS Code) & reset index
county_data.rename(columns={'county_fips':'FIPS Code', 'lat':'Lat', 'lng':'Lng'}, inplace=True)
county_data.sort_values(by='FIPS Code', ascending=True, inplace=True)
county_data.reset_index(inplace=True, drop=True)

## Group coordinates (latitude, longitude) by FIPS Code
county_location = county_data.groupby('FIPS Code').mean()

## Export location to CSV
county_location.to_csv('clean_data/county_coordinates.csv')

## DataFrame preview
county_location

,Lat,Lng
FIPS Code,,
01001,32.562500,-86.547800
01003,30.554945,-87.723960
01005,31.787050,-85.429850
01007,33.059925,-87.145725
01009,33.957085,-86.576808
...,...,...
72145,18.432200,-66.371950
72147,18.124200,-65.461350
72149,18.127900,-66.481400


In [35]:
## Merge census combined and geolocation DataFrames
census_location = census_2019_combined.merge(county_location, how='left', left_index=True, right_index=True)
census_location

,STNAME,CTYNAME,POPESTIMATE2019,TOT_MALE,TOT_FEMALE,0 - 17 years,18 - 49 years,50 - 64 years,65 + years,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Percent non-Hispanic Other,Lat,Lng
FIPS Code,,,,,,,,,,,,,,,,,,
01001,Alabama,Autauga County,55869,27092,28777,14252,21652,11041,8924,0.0283,0.0025,0.0103,0.1900,0.0001,0.7460,0.0228,32.562500,-86.547800
01003,Alabama,Baldwin County,223234,108247,114987,52268,77402,46734,46830,0.0456,0.0065,0.0092,0.0917,0.0000,0.8307,0.0163,30.554945,-87.723960
01005,Alabama,Barbour County,24686,13064,11622,5595,9477,4753,4861,0.0436,0.0029,0.0048,0.4744,0.0000,0.4581,0.0162,31.787050,-85.429850
01007,Alabama,Bibb County,22394,11929,10465,4992,9233,4436,3733,0.0257,0.0013,0.0012,0.2214,0.0000,0.7453,0.0051,33.059925,-87.145725
01009,Alabama,Blount County,57826,28472,29354,14522,21002,11488,10814,0.0926,0.0007,0.0037,0.0153,0.0004,0.8689,0.0184,33.957085,-86.576808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,Wyoming,Sweetwater County,42343,21808,20535,12049,16959,7846,5489,0.1588,0.0102,0.0074,0.0112,0.0003,0.7956,0.0165,41.670024,-109.198271
56039,Wyoming,Teton County,23464,12142,11322,4586,10694,4467,3717,0.1503,0.0033,0.0125,0.0124,0.0012,0.8134,0.0069,43.515844,-110.815633
56041,Wyoming,Uinta County,20226,10224,10002,6215,7229,3757,3025,0.0913,0.0065,0.0016,0.0011,0.0000,0.8752,0.0243,41.293529,-110.539357


In [36]:
## Export Census DataFrame to CSV
census_location.to_csv('analysis_data/census_2019.csv')